In [18]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings

warnings.filterwarnings('ignore')

In [19]:
df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.shape

(7043, 21)

In [20]:
print("Converting TotalCharges to numeric...")
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
nan_count = df['TotalCharges'].isnull().sum()
print(f"   Found {nan_count} missing values")

Converting TotalCharges to numeric...
   Found 11 missing values


In [21]:
# Fill missing values with median
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)
print(f"   Filled with median: {df['TotalCharges'].median():.2f}")
df = df.drop('customerID', axis=1)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
print("   Encoded target: Yes→1, No→0")
X = df.drop('Churn', axis=1)
y = df['Churn']

   Filled with median: 1397.47
   Encoded target: Yes→1, No→0


In [22]:
# Convert categorical variables to numeric codes (simple approach)
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category').cat.codes

print(f"Features shape: {X.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")

Features shape: (7043, 19)
Target distribution: {0: 5174, 1: 1869}


In [23]:
# SPLIT ONCE AND KEEP IT CONSISTENT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features: {X_train.shape[1]}")
print(f"Original training distribution: {y_train.value_counts().to_dict()}")
print(f"Original test distribution: {y_test.value_counts().to_dict()}")

Training set: 5634 samples
Test set: 1409 samples
Features: 19
Original training distribution: {0: 4139, 1: 1495}
Original test distribution: {0: 1035, 1: 374}


In [24]:

# ============ BASELINE MODELS ============
print("\n" + "=" * 50)
print("BASELINE DECISION TREE")
print("=" * 50)

# Baseline fixed model
model_dt = DecisionTreeClassifier(criterion='gini', random_state=100, max_depth=6, min_samples_leaf=8)
model_dt.fit(X_train, y_train)
y_pred_baseline = model_dt.predict(X_test)

print(f"Baseline Accuracy: {accuracy_score(y_test, y_pred_baseline):.4f}")
print(f"Baseline Model Score: {model_dt.score(X_test, y_test):.4f}")


BASELINE DECISION TREE
Baseline Accuracy: 0.7821
Baseline Model Score: 0.7821


In [25]:

# ============ BASELINE WITH HYPERPARAMETER TUNING ============
print("\n" + "=" * 50)
print("BASELINE MODEL - HYPERPARAMETER TUNING")
print("=" * 50)

from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples_leaf': [1, 2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10]
}

# Create and fit tuned model
baseline_tuned = GridSearchCV(
    DecisionTreeClassifier(random_state=100),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

baseline_tuned.fit(X_train, y_train)
y_pred_baseline_tuned = baseline_tuned.predict(X_test)

print(f"Best Parameters: {baseline_tuned.best_params_}")
print(f"Tuned Accuracy: {accuracy_score(y_test, y_pred_baseline_tuned):.4f}")


BASELINE MODEL - HYPERPARAMETER TUNING
Best Parameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}
Tuned Accuracy: 0.7850


In [26]:
# ============ SMOTEENN MODELS WITH PIPELINE ============
print("\n" + "=" * 50)
print("SMOTEENN MODELS (WITH PIPELINE)")
print("=" * 50)

from imblearn.pipeline import Pipeline

# Create pipeline
smote_pipeline = Pipeline([
    ('smoteenn', SMOTEENN(random_state=42)),
    ('classifier', DecisionTreeClassifier(
        criterion='gini',
        random_state=100,
        max_depth=6,
        min_samples_leaf=8
    ))
])

# Train with ONE line (pipeline handles SMOTEENN internally)
smote_pipeline.fit(X_train, y_train)

# Predict with ONE line
y_pred_smote = smote_pipeline.predict(X_test)

print(f"Test Accuracy: {accuracy_score(y_test, y_pred_smote):.4f}")

# Bonus: You can still see what SMOTEENN did
print(f"\nPipeline steps: {[step[0] for step in smote_pipeline.steps]}")


SMOTEENN MODELS (WITH PIPELINE)
Test Accuracy: 0.6977

Pipeline steps: ['smoteenn', 'classifier']


In [27]:
# ============ SMOTEENN TUNED MODEL WITH PIPELINE ============
print("\n--- SMOTEENN Tuned Model (WITH PIPELINE - CORRECT) ---")

from imblearn.pipeline import Pipeline

# Create pipeline first
smote_tuning_pipeline = Pipeline([
    ('smoteenn', SMOTEENN(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=100))
])

# Parameter grid for ENTIRE pipeline
param_grid_smote = {
    'smoteenn__sampling_strategy': [0.5, 0.75, 1.0],  # Control SMOTEENN balance
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_depth': [4, 6, 8, 10, 12],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4, 8],
    'classifier__max_features': ['auto', 'sqrt', 'log2']
}

# GridSearchCV with pipeline
smote_tuned = GridSearchCV(
    smote_tuning_pipeline,  # Use pipeline instead of just model
    param_grid_smote,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=0
)

# Fit on ORIGINAL X_train, y_train (not resampled!)
smote_tuned.fit(X_train, y_train)

print(f"Best Parameters: {smote_tuned.best_params_}")
print(f"Best CV Score: {smote_tuned.best_score_:.4f}")

# Predict with best pipeline
y_pred_smote_tuned = smote_tuned.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_smote_tuned):.4f}")

# Predict on the ORIGINAL test set
y_pred_smote_tuned = smote_tuned.predict(X_test)

print(f"Best Parameters: {smote_tuned.best_params_}")
print(f"Best CV Score: {smote_tuned.best_score_:.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_smote_tuned):.4f}")


--- SMOTEENN Tuned Model (WITH PIPELINE - CORRECT) ---
Best Parameters: {'classifier__criterion': 'gini', 'classifier__max_depth': 12, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 10, 'smoteenn__sampling_strategy': 0.5}
Best CV Score: 0.7865
Test Accuracy: 0.7736
Best Parameters: {'classifier__criterion': 'gini', 'classifier__max_depth': 12, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 10, 'smoteenn__sampling_strategy': 0.5}
Best CV Score: 0.7865
Test Accuracy: 0.7736


In [28]:

# ============ FINAL COMPARISON ============
print("\n" + "=" * 70)
print("FINAL COMPARISON OF ALL MODELS (ALL EVALUATED ON ORIGINAL TEST SET)")
print("=" * 70)

# Prepare results
results = []

# Baseline fixed
acc1 = accuracy_score(y_test, y_pred_baseline)
prec1 = metrics.precision_score(y_test, y_pred_baseline, pos_label=1, zero_division=0)
rec1 = metrics.recall_score(y_test, y_pred_baseline, pos_label=1, zero_division=0)
f1_1 = metrics.f1_score(y_test, y_pred_baseline, pos_label=1, zero_division=0)
results.append(("Baseline (Fixed)", acc1, prec1, rec1, f1_1))

# Baseline tuned
acc2 = accuracy_score(y_test, y_pred_baseline_tuned)
prec2 = metrics.precision_score(y_test, y_pred_baseline_tuned, pos_label=1, zero_division=0)
rec2 = metrics.recall_score(y_test, y_pred_baseline_tuned, pos_label=1, zero_division=0)
f1_2 = metrics.f1_score(y_test, y_pred_baseline_tuned, pos_label=1, zero_division=0)
results.append(("Baseline (Tuned)", acc2, prec2, rec2, f1_2))

# SMOTEENN fixed
acc3 = accuracy_score(y_test, y_pred_smote)
prec3 = metrics.precision_score(y_test, y_pred_smote, pos_label=1, zero_division=0)
rec3 = metrics.recall_score(y_test, y_pred_smote, pos_label=1, zero_division=0)
f1_3 = metrics.f1_score(y_test, y_pred_smote, pos_label=1, zero_division=0)
results.append(("SMOTEENN (Fixed)", acc3, prec3, rec3, f1_3))

# SMOTEENN tuned
acc4 = accuracy_score(y_test, y_pred_smote_tuned)
prec4 = metrics.precision_score(y_test, y_pred_smote_tuned, pos_label=1, zero_division=0)
rec4 = metrics.recall_score(y_test, y_pred_smote_tuned, pos_label=1, zero_division=0)
f1_4 = metrics.f1_score(y_test, y_pred_smote_tuned, pos_label=1, zero_division=0)
results.append(("SMOTEENN (Tuned)", acc4, prec4, rec4, f1_4))

# Display results
print(f"\n{'Model':<25} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
print("-" * 65)

for name, acc, prec, rec, f1 in results:
    print(f"{name:<25} {acc:<10.4f} {prec:<10.4f} {rec:<10.4f} {f1:<10.4f}")

# Find best model
best_acc = max([acc for _, acc, _, _, _ in results])
best_models = [name for name, acc, _, _, _ in results if acc == best_acc]

print(f"\n{'=' * 70}")
print(f"BEST MODEL(S) BY ACCURACY: {', '.join(best_models)} (Accuracy: {best_acc:.4f})")

# Additional: Find best by F1-score (often better for imbalanced data)
best_f1 = max([f1 for _, _, _, _, f1 in results])
best_f1_models = [name for name, _, _, _, f1 in results if f1 == best_f1]
print(f"BEST MODEL(S) BY F1-SCORE: {', '.join(best_f1_models)} (F1: {best_f1:.4f})")

# ============ DETAILED REPORTS ============
print("\n" + "=" * 70)
print("DETAILED CLASSIFICATION REPORTS")
print("=" * 70)

reports = [
    ("Baseline (Fixed)", y_test, y_pred_baseline),
    ("Baseline (Tuned)", y_test, y_pred_baseline_tuned),
    ("SMOTEENN (Fixed)", y_test, y_pred_smote),
    ("SMOTEENN (Tuned)", y_test, y_pred_smote_tuned)
]

for name, y_true, y_pred in reports:
    print(f"\n{name}:")
    print(classification_report(y_true, y_pred, labels=[0, 1], target_names=['No Churn', 'Churn']))
    print(f"Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}")
    print("-" * 50)


FINAL COMPARISON OF ALL MODELS (ALL EVALUATED ON ORIGINAL TEST SET)

Model                     Accuracy   Precision  Recall     F1-Score  
-----------------------------------------------------------------
Baseline (Fixed)          0.7821     0.5971     0.5508     0.5730    
Baseline (Tuned)          0.7850     0.6035     0.5535     0.5774    
SMOTEENN (Fixed)          0.6977     0.4622     0.8503     0.5989    
SMOTEENN (Tuned)          0.7736     0.5733     0.5749     0.5741    

BEST MODEL(S) BY ACCURACY: Baseline (Tuned) (Accuracy: 0.7850)
BEST MODEL(S) BY F1-SCORE: SMOTEENN (Fixed) (F1: 0.5989)

DETAILED CLASSIFICATION REPORTS

Baseline (Fixed):
              precision    recall  f1-score   support

    No Churn       0.84      0.87      0.85      1035
       Churn       0.60      0.55      0.57       374

    accuracy                           0.78      1409
   macro avg       0.72      0.71      0.71      1409
weighted avg       0.78      0.78      0.78      1409

Confusion Matr